In [22]:
import os
from dotenv import load_dotenv
import re
import time
from typing import List, Dict, Any
# import pinecone
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer

load_dotenv()  # Carga las variables del archivo .env

True

In [23]:
# ====== Config Pinecone ======
def configurar_pinecone():
    api_key = os.environ.get("PINECONE_API_KEY")
    if not api_key:
        raise ValueError("PINECONE_API_KEY no configurada")
    pc = Pinecone(api_key=api_key)
    print(f"✅ Pinecone configurado")
    return pc

In [24]:
class GeneradorEmbeddings:
    """
    Clase para generar embeddings usando diferentes modelos.
    """
    
    def __init__(self, modelo: str = "sentence-transformers/all-MiniLM-L6-v2"):
        """
        Inicializa el generador de embeddings.
        
        Args:
            modelo (str): Nombre del modelo de Sentence Transformers
        """
        self.modelo_nombre = modelo
        self.modelo = SentenceTransformer(modelo)
        self.dimension = self.modelo.get_sentence_embedding_dimension()
        
        print(f"✅ Modelo '{modelo}' cargado (dimensión: {self.dimension})")
    
    def generar_embedding(self, texto: str) -> List[float]:
        """
        Genera embedding para un texto individual.
        
        Args:
            texto (str): Texto a convertir en embedding
            
        Returns:
            List[float]: Vector de embedding
        """
        embedding = self.modelo.encode(texto)
        return embedding.tolist()
    
    def generar_embeddings_lote(self, textos: List[str]) -> List[List[float]]:
        """
        Genera embeddings para múltiples textos de manera eficiente.
        
        Args:
            textos (List[str]): Lista de textos
            
        Returns:
            List[List[float]]: Lista de vectores de embedding
        """
        embeddings = self.modelo.encode(textos)
        return [emb.tolist() for emb in embeddings]


In [25]:
# ====== Embeddings ======
class Embeddings:
    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)
        self.dim = self.model.get_sentence_embedding_dimension()
        print(f"✅ Modelo {model_name} (dim={self.dim})")

    def embed_batch(self, texts: List[str]) -> List[List[float]]:
        vecs = self.model.encode(texts, batch_size=32, convert_to_numpy=False)
        return [v.tolist() for v in vecs]

In [26]:
pc = configurar_pinecone()
emb = Embeddings("sentence-transformers/all-MiniLM-L6-v2")

✅ Pinecone configurado
✅ Modelo sentence-transformers/all-MiniLM-L6-v2 (dim=384)
✅ Modelo sentence-transformers/all-MiniLM-L6-v2 (dim=384)


In [27]:
# ================================
# 4. BÚSQUEDAS EN EL ÍNDICE
# ================================

def buscar_documentos_similares(
    nombre_indice: str, 
    consulta: str, 
    generador: GeneradorEmbeddings,
    top_k: int = 3,
    filtro_metadata: Dict = None
) -> List[Dict[str, Any]]:
    """
    Realiza una búsqueda por similitud en el índice.
    
    Args:
        nombre_indice (str): Nombre del índice de Pinecone
        consulta (str): Texto de consulta para buscar
        generador (GeneradorEmbeddings): Generador de embeddings
        top_k (int): Número de resultados más similares a devolver
        filtro_metadata (Dict): Filtros opcionales por metadata
        
    Returns:
        List[Dict]: Lista de documentos similares con scores
    """
    
    # Conectar al índice
    indice = pc.Index(nombre_indice)
    
    # Generar embedding para la consulta
    print(f"🔍 Buscando documentos similares a: '{consulta}'")
    embedding_consulta = generador.generar_embedding(consulta)
    
    # Realizar la búsqueda
    resultados = indice.query(
        vector=embedding_consulta,
        top_k=top_k,
        include_metadata=True
    )

    print(resultados)

    # Procesar y formatear resultados
    documentos_encontrados = []
    
    print(f"\n📋 Resultados encontrados ({len(resultados['matches'])}):")
    print("=" * 80)
    
    for i, match in enumerate(resultados['matches'], 1):
        documento = {
            "posicion": i,
            "id": match["id"],
            "score": round(match["score"], 4),
            "texto": match["metadata"]["text"]
        }
        
        documentos_encontrados.append(documento)
        
        # Mostrar resultado formateado
        print(f"{i}. ID: {documento['id']}")
        print(f"   📊 Score: {documento['score']}")
        print(f"   📝 Texto: {documento['texto'][:100]}...")
        print("-" * 80)
    
    return documentos_encontrados

In [28]:
# Configuración
nombre_indice = "cv-german-384"
# 2. Inicializar generador de embeddings
generador = GeneradorEmbeddings()

# 7. Búsqueda personalizada
print("\n🎯 BÚSQUEDA PERSONALIZADA")
print("=" * 30)
consulta_personalizada = "curso de inglés"
resultados = buscar_documentos_similares(
    nombre_indice, 
    consulta_personalizada, 
    generador,
    top_k=2
)
        
print(f"\n✅ EJEMPLO COMPLETADO EXITOSAMENTE")

✅ Modelo 'sentence-transformers/all-MiniLM-L6-v2' cargado (dimensión: 384)

🎯 BÚSQUEDA PERSONALIZADA
🔍 Buscando documentos similares a: 'curso de inglés'
{'matches': [{'id': 'cv-german::chunk-0003',
              'metadata': {'chunk_id': 3.0,
                           'doc_id': 'cv-german',
                           'len': 1156.0,
                           'text': 'en Tribunal Superior de Justicia de la '
                                   'Provincia de Neuquén, Chos Malal. Junio '
                                   '2016- Febrero 2020 Profesor de Derecho en '
                                   'Instituto Terciario SENECA, Chos Malal. '
                                   'Noviembre 2015- Diciembre 2016 CURSOS:\n'
                                   '\n'
                                   'Curso de Lengua Inglesa nivel "Elementary '
                                   'A"\n'
                                   '\n'
                                   'Curso de Lecto-comprensión en idioma